In [16]:
us_movies.head()

,Release Year,Title,Origin/Ethnicity,Genre,Plot
0,1901,Kansas Saloon Smashers,American,unknown,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,unknown,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,unknown,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,unknown,Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,unknown,The earliest known adaptation of the classic f...


In [ ]:
# Fetch Title and Plot columns
finaldata = us_movies[["Title", "Plot"]]
# Setting the movie title as index
finaldata = finaldata.set_index('Title')
finaldata.head()

In [ ]:

finaldata["Plot"] = finaldata["Plot"].astype(str)
print(finaldata.info())

In [ ]:
#we will need to process data to take out unimportant words
# Download stopwords dataset
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

In [ ]:
finaldata['Processed_Plot'] = finaldata['Plot'].apply(preprocess_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

# Vectorizing the pre-processed plot data using TF-IDF
tfidfvec = TfidfVectorizer() 
tfidf_plot = tfidfvec.fit_transform((finaldata["Processed_Plot"])) 
  
# Find cosine similarity 
from sklearn.metrics.pairwise import cosine_similarity 
cos_sim = cosine_similarity(tfidf_plot, tfidf_plot)

# Storing the indices of the data 
indices = pd.Series(finaldata.index)

In [ ]:
print(tfidf_plot)
#Each row in this matrix corresponds to a document (plot summary), 
# and each column corresponds to a unique word in the entire corpus

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming tfidf_plot is a sparse matrix (scipy.sparse.csr_matrix)
tfidf_dense = tfidf_plot.todense()

# Create a DataFrame for better visualization
tfidf_df = pd.DataFrame(tfidf_dense, columns=tfidfvec.get_feature_names_out(), index=finaldata.index)

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(tfidf_df, cmap='viridis', annot=False)
plt.title('TF-IDF Heatmap for Plot Data')
plt.show()


This is preparing a TF-IDF representation of the movie plot data and then computing the cosine similarity between movies based on their plot descriptions. The indices are stored for later reference. The resulting cos_sim matrix can be used to find similar movies based on their plot content

In [ ]:
def recommendations(title, cosine_sim=cos_sim):
    recommended_movies = []
    index = indices[indices == title].index[0]
    similarity_scores = pd.Series(cosine_sim[index])
    top_10 = list(np.argsort(similarity_scores)[::-1][1:11])
    for i in top_10:
        recommended_movies.append(list(finaldata.index)[i])
    return recommended_movies

In [ ]:
#now lets build the recomendator function

# def recommendations(title, cosine_sim=cos_sim):
#     recommended_movies = []
#     index = indices[indices == title].index[0]
#     similarity_scores = pd.Series(cosine_sim[index].sort_values(ascending=False))
#     top_10 = list(similarity_scores.iloc[1:11].index)
#     for i in top_10:
#         recommended_movies.append(list(finaldata.index)[i])
#     return recommended_movies


In [34]:
recommendations('Little Women')

['Little Women',
 'Little Women',
 'The Ballad of Little Jo',
 'Twister',
 'Frequent Flyer',
 'The Wild Racers',
 'Little Men',
 'See You in the Morning',
 'Funny Face',
 'Vagabond Lady']